In [1]:
#importing neccessary libraires
import keras
from keras import Input, layers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.datasets import imdb

from keras import callbacks
#making the dataset
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,32))


C:\Users\Michael\Anaconda3.2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

#building the model using keras functional API
input_tensor = Input(shape=(64,))
layer1 = layers.Dense(32, activation='relu')(input_tensor)
layer2 = layers.Dense(32, activation='relu')(layer1)
output_tensor = layers.Dense(32, activation='softmax')(layer2)
model = Model(input_tensor, output_tensor)
model.summary()

model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=128)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
Total params: 4,192
Trainable params: 4,192
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1000/1000 [==============================] - 1s 603us/step - loss: 55.3245 - acc: 0.0290
Epoch 2/10
1000/1000 [==============================] - 0s 45us/step - loss: 55.2851 - acc: 0.0320
Epoch 3/10
1000/1000 [==============================] - 0s 45us/step - lo

In [10]:
#building an awesome multi-input model
text_size_vocab = 10000
question_size_vocab = 500
answer_size_vocab = 500

text_input = Input((None,), dtype='int32', name='text')
embedding_layer = layers.Embedding(64, text_size_vocab)(text_input)
encoded_text = layers.LSTM(32)(embedding_layer)

question_input = Input((None,), dtype='int32',name ='question')
embedded_question = layers.Embedding(32, question_size_vocab)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis =-1)
answers = layers.Dense(answer_size_vocab, activation='softmax')(concatenated)

model_2 = Model([text_input, question_input], answers)
model_2.compile('rmsprop',loss='categorical_crossentropy',metrics=['acc'])


model_2.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 500)    16000       question[0][0]                   
__________________________________________________________________________________________________
lstm_5 (LS

In [9]:
samples = 1000
max_features =10
text= np.random.randint(1, text_size_vocab, size = (samples, max_features))
questions = np.random.randint(1, question_size_vocab, size = (samples, max_features))
answers = np.random.randint(0,1,size=(samples, answer_size_vocab))
model_2.fit([text, questions], answers, epochs=10, batch_size=128)

Epoch 1/10


InvalidArgumentError: indices[56,1] = 238 is not in [0, 32)
	 [[Node: embedding_4/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_2/RMSprop/Assign_3"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_4/embeddings/read, _arg_question_1_0_1, training_2/RMSprop/gradients/embedding_4/embedding_lookup_grad/concat/axis)]]

In [5]:
vocabulary_size = 5000
num_income_groups = 10
posts_input = Input(shape=(None,),name='posts')
embedding_posts = layers.Embedding(256,vocabulary_size)(posts_input)
x = layers.Conv1D(128,5,activation='relu')(embedding_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128,activation='relu')(x)

#now the layers that would have to be produced from the main architecture
age_predictions = layers.Dense(1,name='age')(x)
income_prediction = layers.Dense(num_income_groups,activation='softmax',name='income')(x)
gender = layers.Dense(2,activation='softmax',name='gender')(x)

model_3 = Model(posts_input, [age_predictions, income_prediction, gender])
model_3.compile(optimizer='rmsprop',loss={'age':'mse',
                                          'income':'categorical_crossentropy',
                                         'gender':'binary_crossentropy'},
                loss_weights={'age':0.25,'income':1.0,'gender':0.1})

#I didnt run the fit part because this part was only to foster learning
model_3.fit(posts, [age_targets, income_targets, gender_targets], 
            epochs = 10,batch_size=128)

In [6]:
model_3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 5000)   1280000     posts[0][0]                      
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, None, 128)    3200128     embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, None, 128)    0           conv1d_9[0][0]                   
__________________________________________________________________________________________________
conv1d_10 

In [9]:
#Time to write the Xception model
# It is so beautiful
inputer = Input((None,None,3), name='pictures')
branch_a = layers.Conv2D(128,1,activation='relu',strides=2)(inputer)
branch_b = layers.Conv2D(128,1,activation='relu')(inputer)
branch_b = layers.Conv2D(128,3,activation='relu', strides=2)(branch_b)
branch_c = layers.AvgPool2D((3,3),strides=2)(inputer)
branch_c = layers.Conv2D(128,3,activation='relu')(branch_c)
branch_d = layers.Conv2D(128,1,activation='relu')(inputer)
branch_d = layers.Conv2D(128,3,activation='relu')(branch_d)
branch_d = layers.Conv2D(128,3,activation='relu',strides=2)(branch_d)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)
final_layer = layers.Dense(num_classes,)

In [13]:
#learning how to instantiate
lstm = layers.LSTM(32)

left_input = Input((None,128), name='left_input')
left_input = lstm(left_input)

right_input = Input((None, 128), name='right_input')
right_input = lstm(right_input)

merged = layers.concatenate([left_input, right_input], axis = -1)
final_output = layers.Dense(1,activation='sigmoid')(merged)

model_4 = Model([left_input, right_input],final_output)

C:\Users\Michael\Anaconda3.2\lib\site-packages\keras\engine\network.py:186: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer lstm_3.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: lstm_3/TensorArrayReadV3:0
  str(x.name))
C:\Users\Michael\Anaconda3.2\lib\site-packages\keras\engine\network.py:186: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer lstm_3.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: lstm_3_1/TensorArrayReadV3:0
  str(x.nam

AssertionError: 

In [16]:
xception = keras.applications.Xception(include_top=False, weights = None)

In [17]:
left_input = Input((250,250,3))
left_input = xception(left_input)

right_input = Input((250,250,3))
right_input = xception(right_input)
merged = layers.concatenate([left_input, right_input], axis=-1)
predictions = layers.Dense(1,activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions )

C:\Users\Michael\Anaconda3.2\lib\site-packages\keras\engine\network.py:186: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer xception.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: xception/block14_sepconv2_act/Relu:0
  str(x.name))
C:\Users\Michael\Anaconda3.2\lib\site-packages\keras\engine\network.py:186: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer xception.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: xception_1/block14_sepconv

AssertionError: 

In [3]:
x, y = make_classification(n_samples = 10000,n_features = 12,n_classes = 2,weights=[0.6,0.4])

In [4]:
x.shape

(10000, 12)

In [5]:
y.shape

(10000,)

In [8]:
x

array([[ 0.27084756, -1.24426784, -0.6076227 , ..., -0.06653174,
         0.37344524, -0.2708249 ],
       [-0.58553508, -0.97572658, -1.11194589, ..., -0.67305139,
        -0.73403452,  1.07075999],
       [-0.34155723, -1.31244953,  1.76793729, ...,  2.65666615,
        -0.80062065,  0.47199139],
       ...,
       [ 1.46732578, -0.74649137, -1.09923854, ..., -0.62160874,
         0.27439605, -0.04659363],
       [-1.79661379, -0.56753839, -1.30200986, ...,  0.83147013,
        -1.55602165,  2.02348696],
       [-0.25134378, -0.21830916, -1.17936375, ..., -0.01702641,
         1.26344088, -1.12085153]])

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42, test_size=0.25)

In [20]:
callback_list = [callbacks.EarlyStopping(patience=3,monitor='val_acc'),
                callbacks.ModelCheckpoint('model_5.h5',monitor='val_loss',save_best_only=True)]
callback_list_2 = [callbacks.ReduceLROnPlateau(monitor='val_acc',factor = 0.1,patience=5,verbose=1, min_lr=0.0001)]

In [28]:
the_inputer = Input(shape=(12,))
layer_1 = layers.Dense(16,activation='relu')(the_inputer)
layer_2 = layers.Dense(16, activation='relu')(layer_1)
layer_3 = layers.Dense(16, activation='relu')(layer_2)
final_layer = layers.Dense(1,activation='sigmoid')(layer_3)

model_5 = Model(the_inputer, final_layer)
model_5.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model_5_history = model_5.fit(x_train, y_train,batch_size=128,epochs=50,callbacks=callback_list,validation_data=[x_test, y_test])

Train on 7500 samples, validate on 2500 samples
Epoch 1/50
7500/7500 [==============================] - 4s 540us/step - loss: 0.5800 - acc: 0.7093 - val_loss: 0.4787 - val_acc: 0.8396
Epoch 2/50
7500/7500 [==============================] - 0s 44us/step - loss: 0.3547 - acc: 0.8855 - val_loss: 0.2725 - val_acc: 0.9020
Epoch 3/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.2428 - acc: 0.9115 - val_loss: 0.2415 - val_acc: 0.9172
Epoch 4/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.2269 - acc: 0.9141 - val_loss: 0.2344 - val_acc: 0.9184
Epoch 5/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.2209 - acc: 0.9176 - val_loss: 0.2297 - val_acc: 0.9196
Epoch 6/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.2152 - acc: 0.9203 - val_loss: 0.2252 - val_acc: 0.9188
Epoch 7/50
7500/7500 [==============================] - 0s 52us/step - loss: 0.2111 - acc: 0.9221 - val_loss: 0.2226 - val_acc: 0.9184
Epoch 

In [30]:
model_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 12)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 16)                208       
_________________________________________________________________
dense_48 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_49 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 17        
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________


In [33]:
model_5_fit_2 = model_5.fit(x_train, y_train,batch_size=128,epochs=50,callbacks=callback_list_2,validation_data=[x_test, y_test])

Train on 7500 samples, validate on 2500 samples
Epoch 1/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.1972 - acc: 0.9288 - val_loss: 0.2133 - val_acc: 0.9236
Epoch 2/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.1963 - acc: 0.9287 - val_loss: 0.2134 - val_acc: 0.9236
Epoch 3/50
7500/7500 [==============================] - 0s 51us/step - loss: 0.1954 - acc: 0.9300 - val_loss: 0.2117 - val_acc: 0.9256
Epoch 4/50
7500/7500 [==============================] - 0s 46us/step - loss: 0.1946 - acc: 0.9300 - val_loss: 0.2109 - val_acc: 0.9260
Epoch 5/50
7500/7500 [==============================] - 0s 46us/step - loss: 0.1942 - acc: 0.9305 - val_loss: 0.2107 - val_acc: 0.9260
Epoch 6/50
7500/7500 [==============================] - 0s 47us/step - loss: 0.1931 - acc: 0.9319 - val_loss: 0.2097 - val_acc: 0.9264
Epoch 7/50
7500/7500 [==============================] - 0s 49us/step - loss: 0.1920 - acc: 0.9324 - val_loss: 0.2110 - val_acc: 0.9272
Epoch 8

In [2]:
#working on sentiment analyisis with the imbd dataset 
max_features = 2000 # the num of words to take as max features
max_len = 500 #maximun number of length of words
(train_data, train_target),(test_data, test_target) = imdb.load_data(num_words=max_features,maxlen=max_len)

In [3]:
train_data = sequence.pad_sequences(train_data, maxlen=500)
test_data = sequence.pad_sequences(test_data, maxlen=500)

In [4]:
model_6 = Sequential()

In [6]:
model_6.add(layers.Embedding(max_features,128,input_length=max_len,name='embed'))
model_6.add(layers.Conv1D(32,7,activation='relu'))
model_6.add(layers.MaxPooling1D(5))
model_6.add(layers.Conv1D(32,7,activation='relu'))
model_6.add(layers.GlobalAveragePooling1D())
model_6.add(layers.Dense(1))

In [42]:
model_6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [45]:
train_data

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,   28, 1816,   98],
       [   0,    0,    0, ...,  158,   10,   10],
       [   0,    0,    0, ...,  220,  484,  867]])

In [ ]:
callback_list_3 = [callbacks.TensorBoard(log_dir='tensorboard_dir',histogram_freq=1,
                                         embeddings_freq=1,embeddings_data=train_data),
                   callbacks.EarlyStopping(patience=3,monitor='val_acc'),
                callbacks.ModelCheckpoint('model_6.h5',monitor='val_loss',save_best_only=True)]
model_6.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model_6.fit(train_data, train_target,batch_size=32,epochs=40,callbacks = callback_list_3,validation_split=0.25)

Train on 18750 samples, validate on 6250 samples
Epoch 1/40


In [3]:
#writing neural architechtures with depthwise seperable convolutional nets
Inputer_guy = Input((64,64,3))

layer_1 = layers.SeparableConv2D(32,3,activation='relu')(Inputer_guy)
layer_2 = layers.SeparableConv2D(64,3,activation='relu')(layer_1)
layer_3 = layers.MaxPooling2D(2)(layer_2)

layer_4 = layers.SeparableConv2D(64,3,activation='relu')(layer_3)
layer_5 = layers.SeparableConv2D(128,3,activation='relu')(layer_4)
layer_6 = layers.MaxPooling2D(2)(layer_5)

layer_7 = layers.SeparableConv2D(64,3,activation='relu')(layer_6)
layer_8 = layers.SeparableConv2D(128,3,activation='relu')(layer_7)
layer_9 = layers.GlobalAveragePooling2D()(layer_8)

layer_10 = layers.Dense(32,activation='relu')(layer_9)
final_layer = layers.Dense(10,activation='softmax')(layer_10)

model_7 = Model(Inputer_guy, final_layer)
model_7.compile(optimizer='adam',loss='categorical_crossentropy')



In [4]:
model_7.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
separable_conv2d_7 (Separabl (None, 62, 62, 32)        155       
_________________________________________________________________
separable_conv2d_8 (Separabl (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
separable_conv2d_9 (Separabl (None, 28, 28, 64)        4736      
_________________________________________________________________
separable_conv2d_10 (Separab (None, 26, 26, 128)       8896      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 128)       0         
__________